In [1]:
import praw

reddit = praw.Reddit(
    client_id="IISZRvsPo8OBGV06-x_D1Q",
    client_secret="",
    user_agent="Python:SIN project:v1.0 (by /u/Mundane_Trick3335)",
)

Version 7.6.1 of praw is outdated. Version 7.7.0 was released Saturday February 25, 2023.


In [2]:
print(reddit.read_only)

True


In [3]:
for submission in reddit.subreddit("test").hot(limit=10):
    print(submission.title)

test
teyt
This is a test.
Test
Cat and the fly | Nafantik 🐾
This is only another test 2
Test 1
This is only a test 1
test api thank
Showing part of a recent trip report and how I find connections!


In [4]:
prolife_subreddits = [                                         
    "prolife",    
    "MarchForLife",    
    "Abortiondebate",    
    "ProLifeLibertarians",    
    "AbolishAbortion",       
    "StudentsForLife",              
    "RightToLife",                    
    "AbortionRegret",                                
    "RealProLife",           
    "ProLifeMemes",           
    "ProLifeVegan" 
]

In [5]:
prolife_subreddits = [                                         
    "prolife"]

In [6]:
pro_choice_subreddits = [                                                   
    "prochoice",    
    "abortion",    
    "PlannedParenthood",    
    "ReproJustice",       
    "BirthControl",                          
    "LateTermAbortion",
    "AbortionAccess",    
    "MyBodyMyChoice",
    "WomenHealthcare",        
    "reproductivejustice",    
    "ReproductiveRights",        
]

In [7]:
pro_choice_subreddits = [                                                   
    "MarchForLife"]

In [8]:
mixed = [
    "SocialJustice",    
    "Activism", 
    "Progressive",
    "abortiondebate",
    "WomensHealth",
    "Healthcare",
    "Feminism",    
    "FeministTheory",
    "EqualRights",
    "TwoXChromosomes", 
]

In [9]:
import pandas as pd

In [10]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [11]:

tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-stance-abortion')
model = AutoModelForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-stance-abortion')

In [18]:
def relevant_comment(text):
    keywords = ["pro-life", "pro-choice", "abortion", "fetus", "pregnancy", "reproductive rights", "women's rights", "roe v. wade", "contraception", "termination", "embryo", "planned parenthood", "ultrasound", "adoption", "parental", "consent", "incest", "rape", "maternal", "unsafe", "anti-choice", "healthcare access", "education", "teen pregnancy", "ife", "abortion pill", "late-term", "back-alley", "health"]
    new_keywords = ["heartbeat","brain","conception","ban","child", "access", "reproductive", "healthcare", "women's autonomy", "rights","women", "personhood", "sanctity of life", "conscience clause", "abortion restriction", "procedure","quality","child","children","development","functioning"]
    final_keywords=keywords+new_keywords
    for word in keywords:
        if word in text.lower():
            return True
    return False


In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-stance-abortion')
model = AutoModelForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-stance-abortion')

def classify(tok):
    tokens = tokenizer.encode(tok, return_tensors='pt')

    return int(torch.argmax(model(tokens).logits))


def sentiment_score(comment):
    tokens = tokenizer.encode(comment, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [23]:
sentiment_score("My favourite comparison is It's my car, it's my alcohol, I can drive drunk. If you're so worried, drive sober and put on your seatbelt.")

1

In [25]:
file = open('prolife.txt', 'w+')

In [26]:
for subr in prolife_subreddits:
    subreddit = reddit.subreddit(subr)
    print(subreddit.title)
    file.write(subreddit.title+"\n")
    non_echo=0
    choice_echo=0
    life_echo=0
    i=1
    for submission in subreddit.hot(limit=10):
        total=0
        choice=0
        life=0
        # print("Post",i,end=": ")
        file.write("Post "+str(i)+": ")
        tok=submission.title
        if relevant_comment(tok):
            score=sentiment_score(tok[:940])
            total+=1
            if(score==3):
                choice+=1
            elif(score==2):
                life+=1
        # print(submission.author,":")
        submission.comments.replace_more(limit=None)
        
        for top_level_comment in submission.comments:
            tok=top_level_comment.body
            if relevant_comment(tok):
                score=sentiment_score(tok[:940])
                total+=1
                if(score==3):
                    choice+=1
                elif(score==2):
                    life+=1
            # print("\t",top_level_comment.author)
            
            for second_level_comment in top_level_comment.replies:
                tok=second_level_comment.body
                if relevant_comment(tok):
                    score=sentiment_score(tok[:940])
                    total+=1
                    if(score==3):
                        choice+=1
                    elif(score==2):
                        life+=1
        # print(choice, life, total)
        file.write(str(choice)+" "+str(life)+" "+str(total)+"\n")
        i+=1
        if choice>1 and float(total-life)/(float(total)) > 0.8:
            choice_echo+=1
        elif life>1 and float(total-choice)/(float(total)) > 0.8:
            life_echo+=1
        else:
            non_echo+=1 
    print("Overall: ",choice_echo,life_echo,non_echo)
    file.write("Overall: "+str(choice_echo)+" "+ str(life_echo)+" "+str(non_echo)+"\n\n")

ProLife


TypeError: can only concatenate str (not "int") to str

In [41]:
for subr in pro_choice_subreddits:
    subreddit = reddit.subreddit(subr)
    total=0
    choice=0
    life=0
    for submission in subreddit.hot(limit=10):
        print(".",end="")
        tok=submission.title
        if relevant_comment(tok):
            score=sentiment_score(tok[:940])
            total+=1
            if(score==3):
                choice+=1
            elif(score==2):
                life+=1
        # print(submission.author,":")
        submission.comments.replace_more(limit=None)
        
        for top_level_comment in submission.comments:
            tok=top_level_comment.body
            if relevant_comment(tok):
                score=sentiment_score(tok[:940])
                total+=1
                if(score==3):
                    choice+=1
                elif(score==2):
                    life+=1
            # print("\t",top_level_comment.author)
            
            for second_level_comment in top_level_comment.replies:
                tok=second_level_comment.body
                if relevant_comment(tok):
                    score=sentiment_score(tok[:940])
                    total+=1
                    if(score==3):
                        choice+=1
                    elif(score==2):
                        life+=1
    # subreddit = reddit.subreddit(subr)
    print(subreddit.title, choice, life, total)

..........Pro-choice 37 3 44


In [10]:
for subr in mixed:
    subreddit = reddit.subreddit(subr)
    # print(subreddit.title)
    user_list=[]
    for submission in subreddit.hot(limit=10):
        user_list.append(submission.author)
        # print(submission.author,":")
        submission.comments.replace_more(limit=None)
        
        for top_level_comment in submission.comments:
            user_list.append(top_level_comment.author)
            # print("\t",top_level_comment.author)
            
            for second_level_comment in top_level_comment.replies:
                user_list.append(second_level_comment.author)
                # print("\t\t",second_level_comment.author)
    unique_users=list(set(user_list))
    
    interaction_matrix = pd.DataFrame(0, index=unique_users, columns=unique_users)
    for submission in subreddit.hot(limit=10):
    # user_list.append(submission.author)
        submission.comments.replace_more(limit=None)
        
        for top_level_comment in submission.comments:
            interaction_matrix.at[submission.author, top_level_comment.author] = 1
            interaction_matrix.at[top_level_comment.author, submission.author] = 1
            # user_list.append(top_level_comment.author)
            
            for second_level_comment in top_level_comment.replies:
                interaction_matrix.at[second_level_comment.author, top_level_comment.author] = 1
                interaction_matrix.at[top_level_comment.author,second_level_comment.author] = 1
                interaction_matrix.at[second_level_comment.author, submission.author] = 1
                interaction_matrix.at[submission.author,second_level_comment.author] = 1
                # user_list.append(second_level_comment.author)
    # print()

In [11]:
unique_users=list(set(user_list))
print(len(unique_users),len(user_list))

1361 1746


In [12]:
import pandas as pd

In [23]:
interaction_matrix = pd.DataFrame(0, index=unique_users, columns=unique_users)

In [ ]:
for subr in mixed:
    subreddit = reddit.subreddit(subr)
    
    for submission in subreddit.hot(limit=10):
        # user_list.append(submission.author)
        submission.comments.replace_more(limit=None)
        
        for top_level_comment in submission.comments:
            interaction_matrix.at[submission.author, top_level_comment.author] = 1
            interaction_matrix.at[top_level_comment.author, submission.author] = 1
            # user_list.append(top_level_comment.author)
            
            for second_level_comment in top_level_comment.replies:
                interaction_matrix.at[second_level_comment.author, top_level_comment.author] = 1
                interaction_matrix.at[top_level_comment.author,second_level_comment.author] = 1
                interaction_matrix.at[second_level_comment.author, submission.author] = 1
                interaction_matrix.at[submission.author,second_level_comment.author] = 1
                # user_list.append(second_level_comment.author)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
def sentiment_score(comment):
    tokens = tokenizer.encode(comment, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(interaction_matrix)

# print the cluster labels for each user
print(kmeans.labels_)

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[0 0 0 ... 0 0 0]


In [ ]:
sum=0
for i in range(len(unique_users)):
    sum+=interaction_matrix.at[unique_users[32],unique_users[i]]
print(unique_users[32],sum)

cuntliflower 1


In [ ]:
i=0
for subr in mixed:
    subreddit = reddit.subreddit(subr)
    print(subreddit.title)
    for submission in subreddit.hot(limit=10):
        # user_list.append(submission.author)
        print(i,submission.author,":")
        i=i+1
        submission.comments.replace_more(limit=None)
        
        for top_level_comment in submission.comments:
            # user_list.append(top_level_comment.author)
            print("\t",top_level_comment.author)
            i=i+1
            
            for second_level_comment in top_level_comment.replies:
                # user_list.append(second_level_comment.author)
                print("\t\t",second_level_comment.author)
                i=i+1
    print()

Social Justice
0 TheYellowRose :


KeyboardInterrupt: 